In [1]:
from preprocess import *
import os
import ast
import warnings
from utils import *
from collections import Counter
warnings.filterwarnings("ignore", category=DeprecationWarning)
from transfer_learning import *
from eval_NN import eval_avg_score
import glob

In [2]:
kit_label_encoding_cresemba = open_util('sbx-test-12343977','utility_files/cresemba_KITS.txt','txt')
kit_label_decoding_cresemba = dict([(value, key) for key, value in kit_label_encoding_cresemba.items()])

kit_label_encoding_zavicefta = open_util('sbx-test-12343977','utility_files/zavicefta_KITS.txt','txt')
kit_label_decoding_zavicefta = dict([(value, key) for key, value in kit_label_encoding_zavicefta.items()])

insights_label_encoding_cresemba = open_util('sbx-test-12343977','utility_files/cresemba_insights.txt','txt') 
insights_label_decoding_cresemba = dict([(value, key) for key, value in insights_label_encoding_cresemba.items()])

insights_label_encoding_zavicefta = open_util('sbx-test-12343977','utility_files/zavicefta_insights.txt','txt')
insights_label_decoding_zavicefta = dict([(value, key) for key, value in insights_label_encoding_zavicefta.items()])

print(kit_label_encoding_cresemba)
print(kit_label_decoding_cresemba)

{'Not Applicable': 0, 'Safety & PK of antifungals': 1, 'Isavuconazole Life Cycle:Indications, Populations & Formulation': 2, 'Resistance & epidemiology of IFI': 3, 'Awareness & diagnosis of invasive fungal infections (IFI)': 4}
{0: 'Not Applicable', 1: 'Safety & PK of antifungals', 2: 'Isavuconazole Life Cycle:Indications, Populations & Formulation', 3: 'Resistance & epidemiology of IFI', 4: 'Awareness & diagnosis of invasive fungal infections (IFI)'}


In [3]:
def main_pipeline(dataset, n_folds,dict1, eval_test, label, 
                  model_name, model_alias, epochs,
                  batch_size, seed_val,  verbose):

    if label == 'KIT description':
        if dataset == 'cresemba':
            label_encoding = kit_label_encoding_cresemba
            
            dataset_df = preprocess_cresemba_raw_for_kit(dict1)
            print(dataset_df)
        elif dataset == 'zavicefta':
            label_encoding = kit_label_encoding_zavicefta
            
            dataset_df = preprocess_zavicefta_raw_for_kit()
    elif label == 'Insights Category':
        if dataset == 'cresemba':
            label_encoding = insights_label_encoding_cresemba
            
            dataset_df = preprocess_cresemba_raw_for_insights(label_encoding)
        elif dataset == 'zavicefta':
            label_encoding = insights_label_encoding_zavicefta
            
            dataset_df = preprocess_zavicefta_raw_for_insights(label_encoding)

    labels_set = list(label_encoding.keys())
    
    
    biobert_models_path = os.path.join(*[project_path, "data/biobert_model_files"])

    cv_data, _ = cross_val(df=dataset_df, n_folds=int(n_folds), label=label)

    print('Number of data points in each class:')
    labels = cv_data[0][0][label]
    print(Counter(labels))

    cv_data, label_set = pick_data_for_exp_scenario(folds=cv_data, eval_test=eval_test, dataset=dataset, label=label,
                                                    label_mapping=label_encoding, labels_set=labels_set)

    

    best_predictions_list = cross_val_train(cv_data, model_name,
                                            epochs, batch_size, seed_val, 
                                            verbose, biobert_models_path, model_alias)

    metrics = eval_avg_score(best_predictions_list, n_folds, project_path, label, dataset)

In [ ]:
if __name__ == "__main__":

    main_pipeline(dataset = "cresemba",
                    n_folds = 2,
                    eval_test = True,
                    label = "KIT description",
                    
                    # DEFINE PARAMETERS FOR FINETUNING:
                    model_name = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract',
                    model_alias = 'ms_pubmed_bert', #for model saving file name
                    epochs = 1,
                    batch_size = 3,
                    seed_val = 17,
                    dict1=open_util('sbx-test-12343977','Medical Insights_May2021_DFOs_SV_Cresemba_(Isavuconazole).xlsx',file_type='xlsx'),
                    verbose = False,
                    
                    
                    )
    torch.cuda.empty_cache()

     CFM Form ID                                    DFO Description  \
0         178762  there is an unmet need in fungi diagnostics an...   
1         184389  when presenting isavuconazole data in critical...   
2         198269  importance of following patients with severe c...   
3         226329  in paediatric patients, there are difficulties...   
4         242516  3 haematooncology centers in poland have exper...   
...          ...                                                ...   
1731      310451  "the doctor sees 5-10 cases of mm per year and...   
1732      303623  is there are correlation between construction ...   
1733      310431  "the doctor sees 6-10 cases of mm per year and...   
1734      310440  "the doctor sees 5-10 cases of mm per year and...   
1735      224893  in a patient with invasive aspergillosis and t...   

                                        KIT description Insights Category  \
0     Awareness & diagnosis of invasive fungal infec...         Awaren

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSeque